In [ ]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import librosa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def create_dataset(folder_path): 

  images = []
  names = []
  max_length = 0
  for file in os.listdir(folder_path):
    
    image_path = os.path.join(folder_path, file)
    size = Image.open(image_path).size
    image = np.array(Image.open(image_path).resize((128,128)))
    image = image.astype('float32')
    image = np.delete(image, [3], axis=2)
    #Normalization
    image /= 255.0
    images.append(image)
    names.append(file[:-4])
    if image.shape[1]>max_length:
      max_length = image.shape[1]

  for i in range(len(images)):
    images[i] = librosa.util.fix_length(images[i], max_length, axis=1)

  images = np.array(images)

  df = pd.read_csv('/content/drive/MyDrive/Deneme/clotho_metadata_development.csv')

  keywords = []
  for i in names:
    keywords.append(df.keywords[df.file_name == i].to_list()[0].lower().split(';'))

  return names,images,keywords

In [ ]:
names, images, keywords = create_dataset("/content/drive/MyDrive/Dataset Images")

In [ ]:
all_words = []
for i in keywords:
  for j in i:
    if len(j)>2:
      all_words.append(j)

len(all_words)

14371

In [ ]:
from collections import Counter

frequency_list = Counter(all_words).most_common(300)

In [ ]:
keywords_updated = []
for j in range(len(keywords)):
  keyword_list = []
  for i in frequency_list:
    if i[0] in keywords[j]:
      keyword_list.append(i[0])
  keywords_updated.append(keyword_list)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
multihot = mlb.fit_transform(keywords_updated)
class_labels = list(mlb.classes_)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, multihot, test_size=0.20, random_state=33)
print(X_train.shape,X_test.shape)

(1561, 128, 128, 3) (391, 128, 128, 3)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
#reference : https://www.tensorflow.org/tutorials/audio/simple_audio

model = models.Sequential([
                           
    layers.Input(shape=(128,128, 3)),
    # Downsample the input.
    layers.Resizing(32, 32),
    # Normalize.
    layers.BatchNormalization(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(300),
])

# Compile model

model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss = 'categorical_crossentropy',
  metrics = ['accuracy']
)
 
history = model.fit(X_train, y_train, epochs=20, verbose = 1)

Epoch 1/20
49/49 [==============================] - 10s 191ms/step - loss: 40.9855 - accuracy: 0.0096
Epoch 2/20
49/49 [==============================] - 6s 128ms/step - loss: 40.1662 - accuracy: 0.0167
Epoch 3/20
49/49 [==============================] - 5s 106ms/step - loss: 39.7312 - accuracy: 0.0256
Epoch 4/20
49/49 [==============================] - 5s 107ms/step - loss: 37.7255 - accuracy: 0.0359
Epoch 5/20
49/49 [==============================] - 5s 107ms/step - loss: 37.1629 - accuracy: 0.0327
Epoch 6/20
49/49 [==============================] - 5s 107ms/step - loss: 38.3861 - accuracy: 0.0282
Epoch 7/20
49/49 [==============================] - 5s 107ms/step - loss: 37.1822 - accuracy: 0.0320
Epoch 8/20
49/49 [==============================] - 5s 106ms/step - loss: 38.8218 - accuracy: 0.0237
Epoch 9/20
49/49 [==============================] - 5s 107ms/step - loss: 37.7718 - accuracy: 0.0314
Epoch 10/20
49/49 [==============================] - 5s 110ms/step - loss: 36.9956 - accur